<img src="./img/HWNI_logo.svg"/>

# Lab B - Multiple Comparisons and ANOVA

In [1]:
# makes our plots show up inside Jupyter
%matplotlib inline

import numpy as np
import pandas as pd

import scipy.stats

import matplotlib.pyplot as plt
import seaborn as sns

import util.utils as utils
import util.shared as shared

shared.format_plots()
shared.format_dataframes()

/Users/charlesfrye/anaconda3/envs/neur299/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


When N-way ANOVAs are performed in an exploratory fashion (i.e., without pre-specifying which interactions are of interest), there is a 
[rarely-acknowledged multiple-comparisons effect](https://arxiv.org/pdf/1412.3416)
that
[massively increases the error rate](http://deevybee.blogspot.co.uk/2013/06/interpreting-unexpected-significant.html).

Below, we'll repeatedly simulate null-distributed data for an N-way ANOVA and then perform statistical testing. Because we know *a priori* that the null hypothesis is true, we can interpret any finding as a false positive and so get a sense of our error rate.

## Simulating Data

We simulate data by generating a single Gaussian random variable, `outcome`, and then assigning it at random to one of two factor levels in each of a number of factors given by `num_factors`. The factor levels are coded as `0` and `1` and factors are labeled with letters `a-z`. This is done for each of `num_subjects` subjects. The results are returned as a pandas dataframe.

In a neuroscience context, a dataset like this might arise when dealing with human subjects. Say we are interested, so our outcome is some $z$-scored measure of performance on a memory task. It's quite easy to find an unending stream of binary factors that might influence memory: does the subject regularly get 8 or more hours of sleep? are they regular coffee drinkers? are they suffering from a neurodegenerative disease? have they achieved a terminal degree? do they exercise regularly? And so on. The temptation, for the cautious scientist, is to include all of these factors, for fear of missing something. As we shall see, this is not a good idea.

The ANOVA is run using the linear-model-fitting tools in `statsmodels`. The results are also returned as a pandas dataframe. The `p` value is in the column `PR(>F)`.

We can control the power of the study by changing the number of subjects and the standard deviation.

Note that if you're looking at ANOVAs with more factors, you might start coming across linear algebra errors due to the sample size being too low. If that happens, increase the number of subjects.

In [2]:
num_subjects = 1000
num_factors = 7
standard_dev = 1

null_data = utils.generate_data(num_subjects,num_factors,standard_dev)

null_data.tail(10)

,a,b,c,d,e,f,g,outcome
990,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.565795
991,0.0,1.0,1.0,0.0,1.0,0.0,0.0,-0.071758
992,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.311423
993,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.970418
994,0.0,1.0,0.0,1.0,0.0,1.0,0.0,-1.774602
995,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.905214
996,1.0,1.0,0.0,1.0,1.0,1.0,1.0,2.043955
997,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.467722
998,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.235248
999,1.0,1.0,0.0,1.0,0.0,0.0,0.0,-1.625611


In [3]:
results = utils.run_ANOVA(null_data)
len(results)

128

The code below will run this simulated experiment multiple times, tracking three things for each experiment: the fraction of tests that were false positives, whether any false positive occurred, and the result of an "omnibus" test (see question 5 for more information). The results are stored in lists, where the element at index `i` is from the `i`th simulated experiment. Run it with the settings below.

Ignore the `omnibus_test` code for now.

In [4]:
def run_experiments(num_experiments, num_subjects, num_factors, standard_dev):
    
    familywise_errors = np.zeros(num_experiments)
    false_positive_rates = np.zeros(num_experiments)
    omnibus_results = np.zeros(num_experiments)
    
    for experiment in range(num_experiments):
        result = utils.run_ANOVA(utils.generate_data(num_subjects,
                                                     num_factors,
                                                     standard_dev))[:-1]
        false_positive_rates[experiment] = sum(result["PR(>F)"]<0.05)/len(result["PR(>F)"])
        familywise_errors[experiment] = false_positive_rates[experiment]>0
        omnibus_results[experiment] = omnibus_test(result)
        
    return false_positive_rates, familywise_errors, omnibus_results

def omnibus_test(result):
    
    model = result[:-1]
    residual = result.iloc[-1]
    
    dof = np.sum(model['df'])
    
    meansquare_explained = np.sum(model['sum_sq'])/dof
    
    meansquare_unexplained = residual['sum_sq']/residual['df']
    
    F = meansquare_explained/meansquare_unexplained
    
    return (1-scipy.stats.distributions.f.cdf(F,dof,residual['df']))<0.05

In [5]:
num_experiments = 100

num_subjects = 2500
num_factors = 3
standard_dev = 0.5

FPRs, FWERs, omnis = run_experiments(num_experiments, num_subjects, num_factors, standard_dev)

## Simulations

#### Q1 Use your results to calculate both the average family-wise error rate (the chance you get at least one (falsely) signficant result), and the average false positive rate (the fraction of signficant results). Explain your results below.

In [6]:
print(np.mean(FPRs))

print(np.mean(FWERs))

print(np.mean(omnis))

0.0471428571429
0.29
0.03


<font color='1874CD'> **The familywise error rate is around 30% for three factors. The false positive rate remains at 5%. The false positive rate only controls the chance that an individual test is falsely positive, so it's powerless to control the familywise error rate.**

#### Q2 Note that you did not calculate a false discovery rate. What is the FDR for these experiments?

<font color='1874CD'> **The false discovery rate is 100%, so long as there is any discovery at all. Because the null hypothesis is true in every case, there are no true positives. If there are no discoveries, the FDR is undefined.**

#### Q3 What do you predict would be the effect of increasing the number of factors (to, e.g., 7) on the FPR and FWER? Check your prediction against the simulation and report the results.

<font color='1874CD'> ** It should increase the familywise error rate but leave the false positive rate unchanged. **

#### Q4 Make a prediction for the effect of increasing the power (by adding more subjects or decreasing the standard deviation). Check your prediction against the simulation and explain the results.

<font color='1874CD'> ** Though one might expect power to help here, it actually has no effect. Higher power allows us to discover smaller and smaller effects but does not guarantee that any effects we find are less likely to be false. If we find a large effect with a high power, we can be more confident that the result is not due to chance, but that doesn't directly affect the FPR or FWER.**

An "omnibus test" works as follows: first, check whether the model as a whole has a statistically significant mean-square. This can be done by adding the sum-of-squares for each component of the model (except the residuals) and dividing by the sum of the degrees of freedom for each component of the model (again, except the residuals). Then, if the overall result is significant at a level $\alpha$, perform follow-up $F$-tests. This is analogous to how we used one-way ANOVAs to perform follow-up $t$-tests for one-way ANOVAs.

#### Q5 How often would you expect the omnibus test at level $\alpha$ to fail if the null hypothesis is true for all interactions? Check your prediction against the results of the `omnibus_test`.

<font color='1874CD'> **When the null hypothesis is true for all interactions, the omnibus test should fail with the rate $\alpha$. **

#### Q6 What happens to the omnibus test if there is exactly one strong effect in the data? Does it still protect against false positives?

<font color='1874CD'> ** If there is exactly one strong effect in the data, the omnibus test will turn up positive (with a rate given by the power), and we will lose the protection against false positives. That is, we will be unable to tell which of the positives we get are false and which are true. If the prior probability of the hypothesis is small, then these false positives can outweigh the true positives and result in a high false discovery rate. **